In [ ]:
from speos.utils.config import Config
from speos.preprocessing.handler import InputHandler
import os
os.chdir("..")


In [ ]:
import json
from extensions.preprocessing import preprocess_labels
import pandas as pd
def get_coregenes(trait: str, background):
    trait2name = {"uc": "uc",
                "cad": "cad_really",
                "scz": "scz",
                "ad": "alz",
                "ra": "ra"}

    mendelians = preprocess_labels("./extensions/{}_only_genes.tsv".format(trait2name[trait]))

    hsps= pd.read_csv("./hsps/{}.txt".format(trait), header=None, index_col=None).iloc[:, 0].tolist()

    with open("/mnt/storage/speos/results/{}_film_nohetioouter_results.json".format(trait2name[trait]), "r") as file:
        candidate2cs = json.load(file)[0]

    coregenes = [key for key, value in candidate2cs.items() if value == 11]

    other_coregenes = [key for key, value in candidate2cs.items() if value != 11]

    allcore = set()
    allcore.update(set(coregenes))
    allcore.update(set(mendelians))
    allcore = allcore.intersection(set(background))

    noncore = set(background).difference(allcore).difference(other_coregenes)

    return allcore, other_coregenes, hsps, noncore

In [ ]:

trait= "uc"

config = Config()
config.parse_yaml("/home/ubuntu/speos/config_{}_only_nohetio_film_newstorage.yaml".format(trait))
prepro = InputHandler(config).get_preprocessor()
prepro.build_graph(adjacency=False)

In [ ]:
allcore, other_coregenes, hsps,  noncore = get_coregenes(trait, prepro.id2hgnc.values())
mendelians = preprocess_labels("./extensions/{}_only_genes.tsv".format(trait))

In [ ]:
len(allcore) + len(other_coregenes) - prepro.get_data()[1].sum()

In [ ]:
#taken from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9201719/

degs = set(pd.read_csv("deg_uc.tsv", sep=" ", header=None).transpose()[0].tolist())

In [ ]:
from scipy.stats import fisher_exact

all_genes = set()
all_genes.update(allcore)
all_genes.update(other_coregenes)
all_genes.update(noncore)

for group in [set(mendelians), set(allcore).difference(mendelians), set(other_coregenes), set(noncore)]:
    a = len(group.intersection(degs))
    b = len(all_genes.difference(group).intersection(degs))
    c = len(group.difference(degs))
    d = len(all_genes.difference(group).difference(degs))
    array = [[a, b], [c, d]]
    print(fisher_exact(array))

In [ ]:
import scipy.stats as stats


with open("/mnt/storage/speos/results/{}_film_nohetioouter_results.json".format("uc"), "r") as file:
        candidate2cs = json.load(file)[0]

pvals = []
zvals = []
odds_coregenes = []


for cutoff in range(11, 1, -1):
    coregenes = [key for key, value in candidate2cs.items() if value >= cutoff]
    other_coregenes = [key for key, value in candidate2cs.items() if value < cutoff]

    arrays = []
    for group in [set(coregenes).difference(mendelians), set(other_coregenes)]:
        a = len(group.intersection(degs))
        b = len(all_genes.difference(group).intersection(degs))
        c = len(group.difference(degs))
        d = len(all_genes.difference(group).difference(degs))
        arrays.append(np.asarray([[a, b], [c, d]]))

    odds_typical = fisher_exact(arrays[0])[0]
    odds_atypical = fisher_exact(arrays[1])[0]

    odds_coregenes.append(odds_typical)

    log_odds_typical = np.log(odds_typical)
    log_odds_atypical = np.log(odds_atypical)
    delta = log_odds_typical - log_odds_atypical


    var_typical = np.sum([1 / value for value in arrays[0].flatten()])
    var_atypical = np.sum([1 / value for value in arrays[1].flatten()])

    se_delta = np.sqrt(var_typical + var_atypical)
    zval = delta / se_delta
    zvals.append(zval)
    pval = stats.norm.sf(np.abs(zval)) * 2
    pvals.append(pval)

In [ ]:
pvals

In [ ]:
zvals

In [ ]:
odds_coregenes

In [ ]:
fdrcorrection(pvals)

In [ ]:
arrays = []
for group in [set(allcore).difference(mendelians), set(mendelians)]:
    a = len(group.intersection(degs))
    b = len(all_genes.difference(group).intersection(degs))
    c = len(group.difference(degs))
    d = len(all_genes.difference(group).difference(degs))
    arrays.append(np.asarray([[a, b], [c, d]]))

odds_typical = fisher_exact(arrays[0])[0]
odds_atypical = fisher_exact(arrays[1])[0]

odds_coregenes.append(odds_typical)

log_odds_typical = np.log(odds_typical)
log_odds_atypical = np.log(odds_atypical)
delta = log_odds_typical - log_odds_atypical


var_typical = np.sum([1 / value for value in arrays[0].flatten()])
var_atypical = np.sum([1 / value for value in arrays[1].flatten()])

se_delta = np.sqrt(var_typical + var_atypical)
zval = delta / se_delta
zvals.append(zval)
pval = stats.norm.sf(np.abs(zval)) * 2
pval

In [ ]:
coregenes = [key for key, value in candidate2cs.items() if value >= 11]
other_coregenes = [key for key, value in candidate2cs.items() if value < 11]

arrays = []
for group in [set(other_coregenes), set(mendelians).union(coregenes)]:
    a = len(group.intersection(degs))
    b = len(all_genes.difference(group).intersection(degs))
    c = len(group.difference(degs))
    d = len(all_genes.difference(group).difference(degs))
    arrays.append(np.asarray([[a, b], [c, d]]))

odds_typical = fisher_exact(arrays[0])[0]
odds_atypical = fisher_exact(arrays[1])[0]

odds_coregenes.append(odds_typical)

log_odds_typical = np.log(odds_typical)
log_odds_atypical = np.log(odds_atypical)
delta = log_odds_typical - log_odds_atypical


var_typical = np.sum([1 / value for value in arrays[0].flatten()])
var_atypical = np.sum([1 / value for value in arrays[1].flatten()])

se_delta = np.sqrt(var_typical + var_atypical)
zval = delta / se_delta
zvals.append(zval)
pval = stats.norm.sf(np.abs(zval)) * 2
pval

# More analyses with already prepared data

In [ ]:
import pandas as pd

df = pd.read_csv("pc_genes.txt", header=0, sep="\t", index_col=None)

In [ ]:
from speos.visualization.settings import *
import seaborn as sns
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(cm*full_width*0.33, 6*cm), sharey=True)
from matplotlib import ticker

def pval_to_string(pval):
    if pval < 1e-3:
        return "***"
    elif pval < 1e-2:
        return "***"
    elif pval < 5e-2:
        return "***"
    else:
        return "n.s."

for ax, trait in zip(axs, ["uc", "cad", "scz"]):

    
    
    hgnc2tss = {row["gene"]: row["promoter_count"] for i, row in df.iterrows()}
    coregenes, hsps, weakcore, noncore = get_coregenes(trait, background = hgnc2tss.keys())
    from scipy.stats import mannwhitneyu
    stat, p = mannwhitneyu([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in noncore])
    #print(mannwhitneyu([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in weakcore]))
    df2 = pd.DataFrame({"TSS Count": [hgnc2tss[hgnc] for hgnc in coregenes] +  [hgnc2tss[hgnc] for hgnc in noncore],
                        "Group": ["Core Genes\n(n={})".format(len(coregenes))] * len(coregenes)  + ["Peripherals\n(n={})".format(len(noncore))] * len(noncore)})

    #bp = ax.boxplot(x=([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in noncore]), 
    #          positions=range(2), widths=[0.3, 0.3], showfliers=False, zorder=5, patch_artist=True)

    sns.boxplot(df2, x="Group", y="TSS Count", ax=ax, showfliers=False,
                palette={"Core Genes\n(n={})".format(len(coregenes)): "#01016f",
                        "Peripherals\n(n={})".format(len(noncore)):  "#5a5a5a"},
                    linewidth=0.5,
                    width=0.5,
                    medianprops=dict(color="white"))
    if trait != "uc":
        ax.set_ylabel("")
    ax.set_xticks((0,1))
    ax.set_xticklabels(["Core Genes\n(n={})".format(len(coregenes)), "Peripherals\n(n={})".format(len(noncore))], rotation=90)
    ax.set_xlabel("")
    ax.text(x=0.5, y=19, s=trait.upper(), ha="center", va="center", fontsize=8)
    ax.text(x=0.5, y=17, s=pval_to_string(p), ha="center", va="center", fontsize=8)
    ax.set_ylim(-0.1, 20)
    ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

#fig.subplots_adjust(wspace=-3)
plt.tight_layout()
plt.savefig("promoter_count.svg", bbox_inches="tight")

In [ ]:
from speos.visualization.settings import *
import seaborn as sns
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(cm*full_width*0.33, 6*cm), sharey=True)

for ax, trait in zip(axs, ["uc", "cad", "scz"]):
    hgnc2tss = {row["gene"]: row["Roadmap_count"] for i, row in df.iterrows()}
    coregenes, hsps, weakcore, noncore = get_coregenes(trait, background = hgnc2tss.keys())
    from scipy.stats import mannwhitneyu
    stat, p = mannwhitneyu([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in noncore])
    #print(mannwhitneyu([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in weakcore]))
    df2 = pd.DataFrame({"Enhancer Count": [hgnc2tss[hgnc] for hgnc in coregenes] +  [hgnc2tss[hgnc] for hgnc in noncore],
                        "Group": ["Core Genes\n(n={})".format(len(coregenes))] * len(coregenes)  + ["Peripherals\n(n={})".format(len(noncore))] * len(noncore)})

    #bp = ax.boxplot(x=([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in noncore]), 
    #          positions=range(2), widths=[0.3, 0.3], showfliers=False, zorder=5, patch_artist=True)

    sns.boxplot(df2, x="Group", y="Enhancer Count", ax=ax, showfliers=False,
                palette={"Core Genes\n(n={})".format(len(coregenes)): "#01016f",
                        "Peripherals\n(n={})".format(len(noncore)):  "#5a5a5a"},
                    linewidth=0.5,
                    width=0.5,
                    medianprops=dict(color="white"))
    if trait != "uc":
        ax.set_ylabel("")
    ax.set_xticks((0,1))
    ax.set_xticklabels(["Core Genes\n(n={})".format(len(coregenes)), "Peripherals\n(n={})".format(len(noncore))], rotation=90)
    ax.set_xlabel("")
    ax.text(x=0.5, y=122, s=trait.upper(), ha="center", va="center", fontsize=8)
    ax.text(x=0.5, y=110, s=pval_to_string(p), ha="center", va="center", fontsize=8)
    ax.set_ylim(-0.1, 130)
    ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

plt.tight_layout()
plt.savefig("enhancer_count.svg", bbox_inches="tight")

# Test some other influencing variables

In [ ]:
from speos.visualization.settings import *
import seaborn as sns
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(cm*full_width*0.33, 6*cm), sharey=True)
from matplotlib import ticker

def pval_to_string(pval):
    if pval < 1e-3:
        return "***"
    elif pval < 1e-2:
        return "***"
    elif pval < 5e-2:
        return "***"
    else:
        return "n.s."

for ax, trait in zip(axs, ["uc", "cad", "scz"]):

    
    
    hgnc2tss = {row["gene"]: row["ABC_count"] for i, row in df.iterrows()}
    coregenes, hsps, weakcore, noncore = get_coregenes(trait, background = hgnc2tss.keys())
    from scipy.stats import mannwhitneyu
    stat, p = mannwhitneyu([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in noncore])
    #print(mannwhitneyu([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in weakcore]))
    df2 = pd.DataFrame({"TSS Count": [hgnc2tss[hgnc] for hgnc in coregenes] +  [hgnc2tss[hgnc] for hgnc in noncore],
                        "Group": ["Core Genes\n(n={})".format(len(coregenes))] * len(coregenes)  + ["Peripherals\n(n={})".format(len(noncore))] * len(noncore)})

    #bp = ax.boxplot(x=([hgnc2tss[hgnc] for hgnc in coregenes], [hgnc2tss[hgnc] for hgnc in noncore]), 
    #          positions=range(2), widths=[0.3, 0.3], showfliers=False, zorder=5, patch_artist=True)

    sns.boxplot(df2, x="Group", y="TSS Count", ax=ax, showfliers=False,
                palette={"Core Genes\n(n={})".format(len(coregenes)): "#01016f",
                        "Peripherals\n(n={})".format(len(noncore)):  "#5a5a5a"},
                    linewidth=0.5,
                    width=0.5,
                    medianprops=dict(color="white"))
    if trait != "uc":
        ax.set_ylabel("")
    ax.set_xticks((0,1))
    ax.set_xticklabels(["Core Genes\n(n={})".format(len(coregenes)), "Peripherals\n(n={})".format(len(noncore))], rotation=90)
    ax.set_xlabel("")
    ax.text(x=0.5, y=19, s=trait.upper(), ha="center", va="center", fontsize=8)
    ax.text(x=0.5, y=17, s=pval_to_string(p), ha="center", va="center", fontsize=8)
    ax.set_ylim(-0.1, 20)
    ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

#fig.subplots_adjust(wspace=-3)
plt.tight_layout()
#plt.savefig("promoter_count.svg", bbox_inches="tight")

In [ ]:
df

# Get growth of core gene set

In [ ]:
coregene_list = [set()]
individual_numbers = [0]
for trait in ["uc", "cad", "scz", "ra", "ad"]:
    coregenes, _, _, _ = get_coregenes(trait=trait, background=prepro.id2hgnc.values())
    coregene_list.append(set(coregenes))
    individual_numbers.append(len(coregenes))

additive = [sum(individual_numbers[:i+1]) for i in range(len(individual_numbers))]

observed = []
for i, number in enumerate(individual_numbers):
    base_set = coregene_list[0]
    for j in range(i+1):
        base_set = base_set.union(coregene_list[j])
    observed.append(len(base_set))

In [ ]:
additive

In [ ]:
observed

In [ ]:
import random
import numpy as np

background_genes = set(prepro.id2hgnc.keys())

def get_random_sample(geneset, ns, seed):
    returnset = set()
    #random.seed(seed)
    for n in ns:
        genesetB = set(random.sample(list(geneset), n))
        returnset = returnset.union(set([genesetB.pop() for _ in range(n)]))
    return len(returnset)


random_expectation = []
q95 = []
q05 = []
for i in range(len(observed)):
    if i < 2:
        random_expectation.append(observed[i])
        q95.append(observed[i])
        q05.append(observed[i])
    else:
        random_overlaps = [get_random_sample(background_genes, individual_numbers[1:i+1], seed) for seed in range(1000)]
        random_expectation.append(np.mean(random_overlaps))
        q95.append(np.quantile(random_overlaps, q=0.995))
        q05.append(np.quantile(random_overlaps, q=0.005))

In [ ]:
from matplotlib import patches
from speos.visualization.settings import *
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(full_width*cm*0.5, 5*cm))

traits = ["UC", "CAD", "SCZ", "RA", "AD"]
handles = []
handles.extend(ax.plot(range(len(additive)), additive, color="grey", linestyle="--"))

#ax.plot(range(len(random_expectation)), random_expectation,  color="darkgrey")
ax.fill_between(range(len(random_expectation)), q05, q95, color="lightgrey")
handles.append(patches.Patch(color="lightgrey"))
#ax.fill_between(range(len(random_expectation)), random_expectation, ,  color="lightgrey")
ax.plot(range(len(observed)), observed, color="#01016f")
handles.append(patches.Patch(color="#01016f"))
ax.set_xticks(range(len(additive)))
ax.set_xticklabels(["None"] + traits, fontsize=10)
for x in range(2, 6):
    if observed[x] < q05[x]:
        if x == 2:
            offset = 200
        elif x == 3:
            offset = 300
        else:
            offset = 100
        ax.text(x=x, y=observed[x] + offset, s="***", ha="center", va="center", fontsize=8)
ax.grid(axis="y", color="lightgray")
ax.set_xlabel("Core Gene Traits")
ax.set_ylabel("Genes")
ax.legend(handles, labels=["Additive", "Expected Union (99% CI)", "Observed Union"], fontsize=8)
plt.savefig("union_coregenes.svg", bbox_inches="tight")